In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv('../week01/metadata/definitions.csv')

# create a boolean indicator =True if the definition is of "machine learning"
data['ml'] = data['term'] == 'machine learning'

print(data.dtypes)
print(data.head(10))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

L = data["ml"] # labels
corpus = data['definition'] # corpus of reviews in words
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.shape)

## Extract polynomial features

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
Xp = poly.fit_transform(X)
Xp.shape

## Polynomial features with the kernel trick

In [ ]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
rbf_feature = RBFSampler(gamma=1, random_state=1, n_components=200000)
X_features = rbf_feature.fit_transform(X)
clf = SGDClassifier(max_iter=5)
clf.fit(X_features, L)   


In [ ]:
X_features.shape

In [ ]:
# fit logistic regression

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs')
clf.fit(X_features, L)

In [ ]:
# look at performance measures
import my_measures

performance_measures = my_measures.BinaryClassificationPerformance(clf.predict(X_features), L, 'clf')
performance_measures.compute_measures()
print(performance_measures.performance_measures)



In [ ]:
# fit SVM linear classifier
from sklearn import linear_model
sgd = linear_model.SGDClassifier()
sgd.fit(X_features, L)

In [ ]:
performance_measures = my_measures.BinaryClassificationPerformance(sgd.predict(X_features), L, 'clf')
performance_measures.compute_measures()
print(performance_measures.performance_measures)


In [ ]:
# machine learning definitions
# Wikipedia, Expert System, Tech Emergence
ml_defs = ["Machine learning is a field of computer science that gives computers the ability to learn without being explicitly programmed.",
          "Machine learning is an application of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. Machine learning focuses on the development of computer programs that can access data and use it learn for themselves.",
          "Machine Learning is the science of getting computers to learn and act like humans do, and improve their learning over time in autonomous fashion, by feeding them data and information in the form of observations and real-world interactions."]
    
# AI definitions
# Wikipedia, Oxford dictionary
ai_defs = ["Artificial intelligence (AI, also machine intelligence, MI) is intelligence demonstrated by machines, in contrast to the natural intelligence (NI) displayed by humans and other animals.",
          "the theory and development of computer systems able to perform tasks that normally require human intelligence, such as visual perception, speech recognition, decision-making, and translation between languages"]

# Definitions of unrelated things: kitten, piano, widget
other_defs = ["A kitten, also known as a kitty or kitty cat, is a juvenile cat.",
             "The piano is an acoustic, stringed musical instrument invented in Italy by Bartolomeo Cristofori around the year 1700 in which the strings are struck by hammers.",
             "a small gadget or mechanical device, especially one whose name is unknown or unspecified"]
    
def get_prediction(definition):
    text_x = vectorizer.transform([definition]).toarray()
    rbf_feature = RBFSampler(gamma=1, random_state=1, n_components=200000)
    X_features = rbf_feature.fit_transform(text_x)
    return(sgd.predict(X_features))

print("Model predictions for 'machine learning' definitions:")
for mld in ml_defs:
    print(get_prediction(mld))
    
print("* * *")
print("Model predictions for 'AI' definitions:")
for aid in ai_defs:
    print(get_prediction(aid))

print("* * *")
print("Model predictions for other definitions (kitten, piano, widget):")
for otherd in other_defs:
    print(get_prediction(otherd))